In [4]:
"""
=====================================================================
NOTEBOOK 1 : EXPLORATION AVANCÉE DES DONNÉES
Projet : Système de Recommandation MovieLens sur Amazon SageMaker
Auteur : Gninninmaguignon Silué
Date : Octobre 2025
=====================================================================
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os
import urllib.request
import zipfile

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

print("=" * 70)
print("EXPLORATION DES DONNÉES - MOVIELENS 100K")
print("=" * 70)

# ============================================
# PARTIE 1 : TÉLÉCHARGEMENT DU DATASET
# ============================================

print("\nTÉLÉCHARGEMENT DU DATASET MOVIELENS 100K")
print("-" * 70)

# Créer les dossiers si nécessaire
os.makedirs("../data/raw", exist_ok=True)

# URL et chemin
url = "https://files.grouplens.org/datasets/movielens/ml-100k.zip"
zip_path = "../data/raw/ml-100k.zip"
extract_path = "../data/raw"

# Télécharger si pas déjà fait
if not os.path.exists(zip_path):
    print(f"Téléchargement depuis {url}...")
    urllib.request.urlretrieve(url, zip_path)
    print("Téléchargement terminé")
else:
    print("Dataset déjà téléchargé")

# Extraire
if not os.path.exists(f"{extract_path}/ml-100k"):
    print("Extraction du fichier ZIP...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Extraction terminée")
else:
    print("Dataset déjà extrait")

# ============================================
# PARTIE 2 : CHARGEMENT DES DONNÉES
# ============================================

print("\nCHARGEMENT DES FICHIERS")
print("-" * 70)

# Charger les ratings
ratings = pd.read_csv(
    "../data/raw/ml-100k/u.data",
    sep="\t",
    names=["user_id", "item_id", "rating", "timestamp"],
    encoding="latin-1"
)
print(f"Ratings chargés : {ratings.shape[0]:,} lignes")

# Charger les films
movies = pd.read_csv(
    "../data/raw/ml-100k/u.item",
    sep="|",
    encoding="latin-1",
    names=[
        "item_id", "title", "release_date", "video_release_date", "IMDb_URL",
        "unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
        "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
        "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
    ]
)
print(f"Films chargés : {movies.shape[0]:,} films")

# Charger les utilisateurs
users = pd.read_csv(
    "../data/raw/ml-100k/u.user",
    sep="|",
    names=["user_id", "age", "gender", "occupation", "zip_code"],
    encoding="latin-1"
)
print(f"Utilisateurs chargés : {users.shape[0]:,} utilisateurs")

# ============================================
# PARTIE 3 : STATISTIQUES DESCRIPTIVES
# ============================================

print("\n" + "=" * 70)
print("STATISTIQUES DESCRIPTIVES")
print("=" * 70)

print("\n RATINGS")
print("-" * 70)
print(ratings.describe())

print(f"\n Informations clés :")
print(f"  • Nombre total de ratings : {len(ratings):,}")
print(f"  • Nombre d'utilisateurs : {ratings['user_id'].nunique():,}")
print(f"  • Nombre de films : {ratings['item_id'].nunique():,}")
print(f"  • Rating moyen : {ratings['rating'].mean():.2f}")
print(f"  • Rating médian : {ratings['rating'].median():.1f}")
print(f"  • Écart-type : {ratings['rating'].std():.2f}")

# Distribution des ratings
print(f"\n Distribution des ratings :")
rating_dist = ratings['rating'].value_counts().sort_index()
for rating, count in rating_dist.items():
    percentage = (count / len(ratings)) * 100
    print(f"  {rating} ⭐ : {count:,} ({percentage:.1f}%)")

# Sparsité de la matrice
n_users = ratings['user_id'].nunique()
n_items = ratings['item_id'].nunique()
sparsity = (1 - len(ratings) / (n_users * n_items)) * 100
print(f"\n🕸️ Sparsité de la matrice : {sparsity:.2f}%")
print(f"   (Sur {n_users * n_items:,} interactions possibles, seulement {len(ratings):,} existent)")

# ============================================
# PARTIE 4 : ANALYSE DES UTILISATEURS
# ============================================

print("\n" + "=" * 70)
print(" ANALYSE DES UTILISATEURS")
print("=" * 70)

print(f"\n Statistiques démographiques :")
print(f"  • Âge moyen : {users['age'].mean():.1f} ans")
print(f"  • Âge médian : {users['age'].median():.0f} ans")
print(f"  • Âge min/max : {users['age'].min()}-{users['age'].max()} ans")

print(f"\n Répartition par genre :")
gender_dist = users['gender'].value_counts()
for gender, count in gender_dist.items():
    percentage = (count / len(users)) * 100
    label = "Hommes" if gender == "M" else "Femmes"
    print(f"  {label} ({gender}) : {count:,} ({percentage:.1f}%)")

print(f"\n Top 10 professions :")
occupation_dist = users['occupation'].value_counts().head(10)
for occupation, count in occupation_dist.items():
    percentage = (count / len(users)) * 100
    print(f"  {occupation:20s} : {count:3d} ({percentage:.1f}%)")

# Activité des utilisateurs
user_activity = ratings.groupby('user_id').size()
print(f"\n Activité des utilisateurs :")
print(f"  • Ratings par utilisateur (moyenne) : {user_activity.mean():.1f}")
print(f"  • Ratings par utilisateur (médiane) : {user_activity.median():.0f}")
print(f"  • Ratings par utilisateur (min) : {user_activity.min()}")
print(f"  • Ratings par utilisateur (max) : {user_activity.max()}")

# ============================================
# PARTIE 5 : ANALYSE DES FILMS
# ============================================

print("\n" + "=" * 70)
print(" ANALYSE DES FILMS")
print("=" * 70)

# Popularité des films
item_popularity = ratings.groupby('item_id').size()
print(f"\n Popularité des films :")
print(f"  • Ratings par film (moyenne) : {item_popularity.mean():.1f}")
print(f"  • Ratings par film (médiane) : {item_popularity.median():.0f}")
print(f"  • Ratings par film (min) : {item_popularity.min()}")
print(f"  • Ratings par film (max) : {item_popularity.max()}")

# Top films les plus notés
top_rated_items = ratings.groupby('item_id').size().sort_values(ascending=False).head(10)
print(f"\n Top 10 films les plus notés :")
for idx, (item_id, count) in enumerate(top_rated_items.items(), 1):
    movie_title = movies[movies['item_id'] == item_id]['title'].values[0]
    avg_rating = ratings[ratings['item_id'] == item_id]['rating'].mean()
    print(f"  {idx:2d}. {movie_title:45s} : {count:3d} ratings (avg: {avg_rating:.2f}⭐)")

# Genres des films
genre_columns = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy',
                 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
                 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                 'Thriller', 'War', 'Western']

genre_counts = movies[genre_columns].sum().sort_values(ascending=False)
print(f"\n Répartition par genres :")
for genre, count in genre_counts.head(10).items():
    percentage = (count / len(movies)) * 100
    print(f"  {genre:15s} : {count:3d} films ({percentage:.1f}%)")

# ============================================
# PARTIE 6 : VISUALISATIONS
# ============================================

print("\n" + "=" * 70)
print(" GÉNÉRATION DES VISUALISATIONS")
print("=" * 70)

# Créer le dossier de sortie
os.makedirs("../outputs/plots", exist_ok=True)

# Figure 1 : Distribution des ratings
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Analyse Exploratoire - MovieLens 100K', fontsize=16, fontweight='bold')

# 1.1 Distribution des ratings
ax = axes[0, 0]
rating_dist.plot(kind='bar', ax=ax, color='steelblue', edgecolor='black')
ax.set_title('Distribution des Ratings', fontweight='bold')
ax.set_xlabel('Rating (étoiles)')
ax.set_ylabel('Nombre de ratings')
ax.grid(axis='y', alpha=0.3)

# 1.2 Distribution de l'âge des utilisateurs
ax = axes[0, 1]
users['age'].hist(bins=30, ax=ax, color='coral', edgecolor='black')
ax.set_title('Distribution de l\'Âge des Utilisateurs', fontweight='bold')
ax.set_xlabel('Âge')
ax.set_ylabel('Nombre d\'utilisateurs')
ax.grid(axis='y', alpha=0.3)

# 1.3 Activité des utilisateurs
ax = axes[1, 0]
user_activity.hist(bins=50, ax=ax, color='mediumseagreen', edgecolor='black')
ax.set_title('Distribution de l\'Activité des Utilisateurs', fontweight='bold')
ax.set_xlabel('Nombre de ratings par utilisateur')
ax.set_ylabel('Nombre d\'utilisateurs')
ax.set_xlim(0, 300)
ax.grid(axis='y', alpha=0.3)

# 1.4 Popularité des films
ax = axes[1, 1]
item_popularity.hist(bins=50, ax=ax, color='mediumpurple', edgecolor='black')
ax.set_title('Distribution de la Popularité des Films', fontweight='bold')
ax.set_xlabel('Nombre de ratings par film')
ax.set_ylabel('Nombre de films')
ax.set_xlim(0, 300)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/plots/01_exploration_overview.png', dpi=150, bbox_inches='tight')
print(" Graphique sauvegardé : outputs/plots/01_exploration_overview.png")
plt.close()

# Figure 2 : Genres et démographie
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 2.1 Top genres
ax = axes[0]
genre_counts.head(10).plot(kind='barh', ax=ax, color='teal', edgecolor='black')
ax.set_title('Top 10 Genres de Films', fontweight='bold')
ax.set_xlabel('Nombre de films')
ax.grid(axis='x', alpha=0.3)

# 2.2 Répartition hommes/femmes
ax = axes[1]
gender_labels = ['Hommes', 'Femmes']
gender_values = [gender_dist['M'], gender_dist['F']]
colors = ['#3498db', '#e74c3c']
ax.pie(gender_values, labels=gender_labels, autopct='%1.1f%%', colors=colors, startangle=90)
ax.set_title('Répartition par Genre', fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/plots/02_genres_demographie.png', dpi=150, bbox_inches='tight')
print(" Graphique sauvegardé : outputs/plots/02_genres_demographie.png")
plt.close()

# ============================================
# PARTIE 7 : FUSION DES DONNÉES
# ============================================

print("\n" + "=" * 70)
print("🔗 FUSION DES DONNÉES")
print("=" * 70)

# Fusionner toutes les informations
data_full = ratings.copy()
data_full = data_full.merge(movies[['item_id', 'title'] + genre_columns], on='item_id', how='left')
data_full = data_full.merge(users, on='user_id', how='left')

print(f" Dataset fusionné : {data_full.shape}")
print(f"   Colonnes : {data_full.shape[1]}")
print(f"   Lignes : {data_full.shape[0]:,}")

# Aperçu
print("\n Aperçu du dataset fusionné :")
print(data_full.head())

# Vérifier les valeurs manquantes
print("\n🔍 Vérification des valeurs manquantes :")
missing = data_full.isnull().sum()
if missing.sum() == 0:
    print(" Aucune valeur manquante détectée")
else:
    print(missing[missing > 0])

# ============================================
# PARTIE 8 : SAUVEGARDE
# ============================================

print("\n" + "=" * 70)
print(" SAUVEGARDE DES DONNÉES")
print("=" * 70)

# Sauvegarder le dataset complet
os.makedirs("../data/processed", exist_ok=True)
data_full.to_csv("../data/processed/movielens_complete.csv", index=False)
print(" Dataset complet sauvegardé : data/processed/movielens_complete.csv")

# Sauvegarder les métadonnées séparées
movies.to_csv("../data/processed/movies_metadata.csv", index=False)
users.to_csv("../data/processed/users_metadata.csv", index=False)
print(" Métadonnées sauvegardées")

# Créer un rapport JSON
report = {
    'date_analysis': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset': 'MovieLens 100K',
    'statistics': {
        'n_ratings': int(len(ratings)),
        'n_users': int(ratings['user_id'].nunique()),
        'n_items': int(ratings['item_id'].nunique()),
        'avg_rating': float(ratings['rating'].mean()),
        'sparsity': float(sparsity),
        'min_ratings_per_user': int(user_activity.min()),
        'max_ratings_per_user': int(user_activity.max()),
        'avg_ratings_per_user': float(user_activity.mean())
    }
}

import json
with open('../outputs/metrics/exploration_report.json', 'w') as f:
    json.dump(report, f, indent=2)
print(" Rapport JSON sauvegardé : outputs/metrics/exploration_report.json")

# ============================================
# RÉSUMÉ FINAL
# ============================================

print("\n" + "=" * 70)
print(" EXPLORATION TERMINÉE AVEC SUCCÈS")
print("=" * 70)

print("\n RÉSUMÉ DES INSIGHTS :")
print(f"  1. Dataset : {len(ratings):,} ratings de {n_users} utilisateurs sur {n_items} films")
print(f"  2. Sparsité élevée : {sparsity:.2f}% → Besoin de collaborative filtering")
print(f"  3. Distribution biaisée : 75% des ratings sont 3⭐, 4⭐ ou 5⭐")
print(f"  4. Long tail : Certains films très populaires, beaucoup de films peu notés")
print(f"  5. Démographie : {(gender_dist['M']/len(users)*100):.0f}% d'hommes, âge moyen {users['age'].mean():.0f} ans")

print("\n PROCHAINE ÉTAPE : Preprocessing et Feature Engineering")
print("=" * 70)

EXPLORATION DES DONNÉES - MOVIELENS 100K

TÉLÉCHARGEMENT DU DATASET MOVIELENS 100K
----------------------------------------------------------------------
Dataset déjà téléchargé
Dataset déjà extrait

CHARGEMENT DES FICHIERS
----------------------------------------------------------------------
Ratings chargés : 100,000 lignes
Films chargés : 1,682 films
Utilisateurs chargés : 943 utilisateurs

STATISTIQUES DESCRIPTIVES

 RATINGS
----------------------------------------------------------------------
            user_id        item_id         rating     timestamp
count  100000.00000  100000.000000  100000.000000  1.000000e+05
mean      462.48475     425.530130       3.529860  8.835289e+08
std       266.61442     330.798356       1.125674  5.343856e+06
min         1.00000       1.000000       1.000000  8.747247e+08
25%       254.00000     175.000000       3.000000  8.794487e+08
50%       447.00000     322.000000       4.000000  8.828269e+08
75%       682.00000     631.000000       4.00000